__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-11-15 13:06:53 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-15 13:06:53 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-15 13:06:55 | AdsListParser: downloader init
[INFO    ] 2022-11-15 13:06:55 | AvitoParser: init
[INFO    ] 2022-11-15 13:07:00 | AvitoParser: 72 pages for read
[INFO    ] 2022-11-15 13:07:00 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-15 13:07:02 | AdsListParser: read page 1
[INFO    ] 2022-11-15 13:07:05 | AdsListParser: read page 2
[INFO    ] 2022-11-15 13:07:08 | AdsListParser: read page 3
[INFO    ] 2022-11-15 13:07:12 | AdsListParser: read page 4
[INFO    ] 2022-11-15 13:07:16 | AdsListParser: read page 5
[INFO    ] 2022-11-15 13:07:21 | AdsListParser: read page 6
[INFO    ] 2022-11-15 13:07:25 | AdsListParser: read page 7
[INFO    ] 2022-11-15 13:07:27 | AdsListParser: read page 8
[INFO    ] 2022-11-15 13:07:30 | AdsListParser: read page 9
[INFO    ] 2022-11-15 13:07:35 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3579
3218


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
2518,2425338071,"Объявление «2-к. квартира, 57 м², 2/10 эт.» 40 фотографий",6800000,,"ул. Горпищенко, 127к1","Двухкомнатная квартира 57 м2, на 2/10 этаже дома с просторной кухней 10,7 м2. Расположена в г. Севастополь, Нахимовский район, ул. Горпищенко, д. 127 к. 1. \n\nПозвоните нам, чтобы узнать все детали! \n\nТехнические характеристики: \n— Адрес: г. Севастополь, На",57,2022-11-15 13:12:30.595599,sevastopol
235,2601331878,"Объявление «1-к. квартира, 38 м², 3/5 эт.» 17 фотографий",3800000,,"ул. Колобова, 22","Срочная Продажа! Продам небольшую двушку в одном из самых востребованных районе города. Квартира находится непосредственно на территории части Мчс, что несомненно дает вам преимущество(бесплатная охраняемая парковка, круглосуточная охрана части). В квартир",5,2022-11-15 13:12:30.595599,sevastopol


In [11]:
# !pip install dash_table